In [29]:
import os
from Classes.MetaDataCSVLoader import MetaDataCSVLoader
from Classes.OpenAIClient import OpenAIClient
from Classes.PubMedClient import PubMedClient
from utils.constants import keywords, study_designs, system_template, metadata_columns
from utils.utils import create_df_fill_full_abstract, extract_keywords_from_results
import pandas as pd


In [32]:

# Process the data
# openai = OpenAIClient()
pubmed = PubMedClient()

# Retrieving results from PubMed as a df
results = pubmed.search('fever AND covid', max_results=1000)
id_list = results['IdList']
papers = pubmed.fetch_details(id_list)
df = create_df_fill_full_abstract(papers)
# Columns = 'PMID', 'Title', 'Abstract', 'Journal', 'Language', 'Year', 'Month', 'Abstract_1'

# Extracting only the PMID and Abstract
df_abstracts = pd.DataFrame(df, columns=['PMID', 'Abstract'])
display(df_abstracts)

# check if data folder exists, if not create it
if not os.path.exists('data'):
    os.makedirs('data')

# save to csv
df_abstracts.to_csv('data/pubmed.csv', index=False)


# load the csv into a list of documents
loader = MetaDataCSVLoader(file_path="data/pubmed.csv", metadata_columns=metadata_columns, encoding="utf-8")
bg_data = loader.load()

# Create an instance of the OpenAIClient using the data
openai = OpenAIClient(bg_data)



,PMID,Abstract
0,33222680,[COVID-19 is one of the most disastrous respir...
1,34117360,"Of all the suspected infected, 2596 tested neg..."
2,33797605,"[In December 2019, a novel coronavirus disease..."
3,37399831,[Previous SARS-CoV-2 infection and vaccination...
4,35545266,No Abstract
...,...,...
995,37474524,[This study evaluates to what extent symptoms ...
996,38335209,"The prevalence of malaria by microscopy, RDT a..."
997,35129047,[Motivated by the up to now disastrous outcome...
998,32971511,"Of the total of 32 COVID-19 patients with T1D,..."


In [33]:
query_los = "What are the outcomes of covid?"

openai.print_result(query_los, system_template)

Loading DB


> Entering new RetrievalQA chain...

> Finished chain.

        ### Question: 
        What are the outcomes of covid?

        ### Response: 
        The outcomes of COVID-19 can vary widely. The majority of cases recover from the disease without clinical intervention. However, approximately 20% of global cases result in more severe outcomes, such as shortness of breath and pneumonia, necessitating hospitalisation and the requirement of additional oxygen or ventilation. Severe or fatal outcomes are generally more common in older individuals and those with underlying health conditions.

        ### Reference Links:
        https://pubmed.ncbi.nlm.nih.gov/33499798/
        


{'query': 'What are the outcomes of covid?',
 'result': 'The outcomes of COVID-19 can vary widely. The majority of cases recover from the disease without clinical intervention. However, approximately 20% of global cases result in more severe outcomes, such as shortness of breath and pneumonia, necessitating hospitalisation and the requirement of additional oxygen or ventilation. Severe or fatal outcomes are generally more common in older individuals and those with underlying health conditions.',
 'source_documents': [Document(page_content='before symptoms develop until two weeks after symptom onset.3,5 The predominant symptoms reported in COVID-19 cases are cough, sore throat, fatigue, runny nose and fever.6 The majority of cases recover from the disease without clinical intervention; however, approximately 20% of global cases result in more severe outcomes, such as shortness of breath and pneumonia, necessitating hospitalisation and the requirement of additional oxygen or ventilation.

In [11]:
result["source_documents"]

[Document(page_content="Abstract_1: ['Fever is one of the typical symptoms of coronavirus disease (COVID-19). We aimed to investigate the association between early fever (EF) and clinical outcomes in COVID-19 patients. A total of 1,014 COVID-19 patients at the Leishenshan Hospital were enrolled and classified into the EF and non-EF groups based on whether they had fever within 5 days of symptom onset. Risk factors for clinical outcomes in patients with different levels of disease severity were analyzed using", metadata={'Outcomes_from_results': '', 'PMID': '34513787', 'Study_design': '', 'row': 205, 'source': 'data/pubmed.csv'}),
 Document(page_content="Abstract_1: ['Fever is one of the typical symptoms of coronavirus disease (COVID-19). We aimed to investigate the association between early fever (EF) and clinical outcomes in COVID-19 patients. A total of 1,014 COVID-19 patients at the Leishenshan Hospital were enrolled and classified into the EF and non-EF groups based on whether they

In [12]:
query_los = "What is the treatment for covid?"

result = openai.print_result(query_los, system_template)
print(result["source_documents"])



> Entering new RetrievalQA chain...

> Finished chain.

        ### Question: 
        What is the treatment for covid?

        ### Response: 
        As of now, there are several treatments available for COVID-19. These include:

1. Antiviral or immune modulator medications: These are drugs that can help the body's immune system fight the virus or can interfere with the virus's ability to reproduce. Examples include Remdesivir and Dexamethasone.

2. Monoclonal antibodies: These are laboratory-made proteins that mimic the immune system's ability to fight off harmful pathogens such as viruses. Examples include Bamlanivimab and Casirivimab/Imdevimab.

3. Vaccines: Vaccines help prevent people from getting COVID-19 by creating an antibody response without causing the disease. Examples include Pfizer-BioNTech, Moderna, Johnson & Johnson’s Janssen, and AstraZeneca/Oxford vaccines.

4. Supportive care: This includes rest, hydration, and over-the-counter remedies to help with symptoms. In 

In [13]:
for doc in result["source_documents"]:
    print(doc.metadata)
    print(doc.page_content)

    pmid = doc.metadata["PMID"]
    link = f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/"
    print(link)
    print("\n\n")

{'Outcomes_from_results': '', 'PMID': '33303302', 'Study_design': '', 'row': 21, 'source': 'data/pubmed.csv'}
treatments has the potential to prevent COVID-19 and to decrease the severity of mild and moderate cases of Coronavirus. We propose heat treatments for this uncontrolled worldwide coronavirus pandemic while studies are being done to test the effectiveness of heat treatments in the prevention and treatment of COVID-19.']
https://pubmed.ncbi.nlm.nih.gov/33303302/



{'Outcomes_from_results': '', 'PMID': '33303302', 'Study_design': '', 'row': 21, 'source': 'data/pubmed.csv'}
treatments has the potential to prevent COVID-19 and to decrease the severity of mild and moderate cases of Coronavirus. We propose heat treatments for this uncontrolled worldwide coronavirus pandemic while studies are being done to test the effectiveness of heat treatments in the prevention and treatment of COVID-19.']
https://pubmed.ncbi.nlm.nih.gov/33303302/



{'Outcomes_from_results': '', 'PMID': '3330330

In [14]:
query_los = "Wwhat is the effect of covid on the reproductive system?"

result = openai.print_result(query_los, system_template)
print(result["source_documents"])



> Entering new RetrievalQA chain...

> Finished chain.

        ### Question: 
        Wwhat is the effect of covid on the reproductive system?

        ### Response: 
        As an AI, I can provide information based on available data. Some studies suggest that COVID-19 may potentially affect the reproductive system, but the extent and permanence of this impact are still under investigation. In men, some reports indicate that the virus can affect sperm quality and potentially lead to decreased fertility. In women, the virus may impact menstrual cycles and potentially affect fertility, but more research is needed. Pregnant women with COVID-19 may be at a higher risk for severe illness compared to non-pregnant women, and there may be an increased risk of adverse pregnancy outcomes, such as preterm birth. However, it's important to note that research is ongoing and these findings are preliminary. Always consult with a healthcare provider for medical advice.

        ### Reference Links

In [15]:
for doc in result["source_documents"]:
    print(doc.metadata)
    print(doc.page_content)
    print("\n\n")

{'Outcomes_from_results': '', 'PMID': '33528827', 'Study_design': '', 'row': 99, 'source': 'data/pubmed.csv'}
increased risk of CFS. Our study suggested that medical institutions should pay attention to the psychological consequences of the COVID-19 outbreak.']



{'Outcomes_from_results': '', 'PMID': '33528827', 'Study_design': '', 'row': 99, 'source': 'data/pubmed.csv'}
increased risk of CFS. Our study suggested that medical institutions should pay attention to the psychological consequences of the COVID-19 outbreak.']



{'Outcomes_from_results': '', 'PMID': '33528827', 'Study_design': '', 'row': 99, 'source': 'data/pubmed.csv'}
increased risk of CFS. Our study suggested that medical institutions should pay attention to the psychological consequences of the COVID-19 outbreak.']



{'Outcomes_from_results': '', 'PMID': '33528827', 'Study_design': '', 'row': 127, 'source': 'data/pubmed.csv'}
increased risk of CFS. Our study suggested that medical institutions should pay attention to t

In [24]:
client = PubMedClient()
results = client.search('fever AND covid', max_results=10)
id_list = results['IdList']
papers = client.fetch_details(id_list)
print(papers)
df = create_df_fill_full_abstract(papers)
display(df)

extract_abstract = extract_keywords_from_results(df, keywords, study_designs)
display(extract_abstract)

{'PubmedArticle': [{'MedlineCitation': DictElement({'OtherID': [], 'KeywordList': [ListElement([StringElement('CNS', attributes={'MajorTopicYN': 'N'}), StringElement('COVID-19', attributes={'MajorTopicYN': 'N'}), StringElement('PNS', attributes={'MajorTopicYN': 'N'}), StringElement('cerebrovascular disease', attributes={'MajorTopicYN': 'N'}), StringElement('neurological manifestations', attributes={'MajorTopicYN': 'N'}), StringElement('respiratory diseases.', attributes={'MajorTopicYN': 'N'})], attributes={'Owner': 'NOTNLM'})], 'SpaceFlightMission': [], 'GeneralNote': [], 'OtherAbstract': [], 'InvestigatorList': [], 'CitationSubset': ['IM'], 'PMID': StringElement('33222680', attributes={'Version': '1'}), 'DateCompleted': {'Year': '2021', 'Month': '06', 'Day': '16'}, 'DateRevised': {'Year': '2021', 'Month': '06', 'Day': '16'}, 'Article': DictElement({'ArticleDate': [], 'ELocationID': [StringElement('10.2174/1871527319666201120143654', attributes={'EIdType': 'doi', 'ValidYN': 'Y'})], 'La

,PMID,Title,Abstract,Journal,Language,Year,Month,Abstract_1
0,33222680,COVID-19 Outbreak: Neurological Manifestations...,[COVID-19 is one of the most disastrous respir...,CNS & neurological disorders drug targets,eng,2021,No Data,[COVID-19 is one of the most disastrous respir...
1,34117360,Diagnosis of COVID-19 in children guided by la...,"Of all the suspected infected, 2596 tested neg...",Pediatric research,eng,2022,Apr,[The objective of this study is to test how ce...
2,33797605,A systematic review of pregnant women with COV...,"[In December 2019, a novel coronavirus disease...",Archives of gynecology and obstetrics,eng,2021,Jul,"[In December 2019, a novel coronavirus disease..."
3,37399831,Associations of COVID-19 symptoms with omicron...,[Previous SARS-CoV-2 infection and vaccination...,The Lancet. Infectious diseases,eng,2023,Nov,[Previous SARS-CoV-2 infection and vaccination...
4,35545266,"Unresolving fever, headache, cough, and negati...",No Abstract,BMJ (Clinical research ed.),eng,2022,May,No Abstract
5,33231411,Fever management in COVID-19 patients.,No Abstract,Minerva anestesiologica,eng,2021,Jan,No Abstract
6,33546881,COVID-19-Associated Suspected Myocarditis as t...,[Current reports concerning cardiac involvemen...,The American journal of the medical sciences,eng,2021,Apr,[Current reports concerning cardiac involvemen...
7,32804037,"COVID-19, fever and dressings used for pressur...",No Abstract,Journal of wound care,eng,2020,Aug,No Abstract
8,34787753,Parkinson's disease and COVID-19: a systematic...,We found 1693 articles by literature search; a...,Neurological sciences : official journal of th...,eng,2022,Feb,[Patients with Parkinson's disease (PD) are at...
9,33973187,COVID-19 and Tropical Infection: Complexity an...,[COVID-19 is a newly emerging pandemic caused ...,Advances in experimental medicine and biology,eng,2021,No Data,[COVID-19 is a newly emerging pandemic caused ...


,PMID,Title,Abstract,Journal,Language,Year,Month,Abstract_1,Outcomes,Outcomes_from_results,Study_design
0,33222680,COVID-19 Outbreak: Neurological Manifestations...,[COVID-19 is one of the most disastrous respir...,CNS & neurological disorders drug targets,eng,2021,No Data,[COVID-19 is one of the most disastrous respir...,,,
1,34117360,Diagnosis of COVID-19 in children guided by la...,"Of all the suspected infected, 2596 tested neg...",Pediatric research,eng,2022,Apr,[The objective of this study is to test how ce...,,,
2,33797605,A systematic review of pregnant women with COV...,"[In December 2019, a novel coronavirus disease...",Archives of gynecology and obstetrics,eng,2021,Jul,"[In December 2019, a novel coronavirus disease...",,,systematic review
3,37399831,Associations of COVID-19 symptoms with omicron...,[Previous SARS-CoV-2 infection and vaccination...,The Lancet. Infectious diseases,eng,2023,Nov,[Previous SARS-CoV-2 infection and vaccination...,,,
4,35545266,"Unresolving fever, headache, cough, and negati...",No Abstract,BMJ (Clinical research ed.),eng,2022,May,No Abstract,,,
5,33231411,Fever management in COVID-19 patients.,No Abstract,Minerva anestesiologica,eng,2021,Jan,No Abstract,,,
6,33546881,COVID-19-Associated Suspected Myocarditis as t...,[Current reports concerning cardiac involvemen...,The American journal of the medical sciences,eng,2021,Apr,[Current reports concerning cardiac involvemen...,,,
7,32804037,"COVID-19, fever and dressings used for pressur...",No Abstract,Journal of wound care,eng,2020,Aug,No Abstract,,,
8,34787753,Parkinson's disease and COVID-19: a systematic...,We found 1693 articles by literature search; a...,Neurological sciences : official journal of th...,eng,2022,Feb,[Patients with Parkinson's disease (PD) are at...,,mortality,"systematic review, meta-analysis"
9,33973187,COVID-19 and Tropical Infection: Complexity an...,[COVID-19 is a newly emerging pandemic caused ...,Advances in experimental medicine and biology,eng,2021,No Data,[COVID-19 is a newly emerging pandemic caused ...,,,


In [17]:
papers

{'PubmedArticle': [{'MedlineCitation': DictElement({'OtherID': [], 'KeywordList': [ListElement([StringElement('CNS', attributes={'MajorTopicYN': 'N'}), StringElement('COVID-19', attributes={'MajorTopicYN': 'N'}), StringElement('PNS', attributes={'MajorTopicYN': 'N'}), StringElement('cerebrovascular disease', attributes={'MajorTopicYN': 'N'}), StringElement('neurological manifestations', attributes={'MajorTopicYN': 'N'}), StringElement('respiratory diseases.', attributes={'MajorTopicYN': 'N'})], attributes={'Owner': 'NOTNLM'})], 'SpaceFlightMission': [], 'GeneralNote': [], 'OtherAbstract': [], 'InvestigatorList': [], 'CitationSubset': ['IM'], 'PMID': StringElement('33222680', attributes={'Version': '1'}), 'DateCompleted': {'Year': '2021', 'Month': '06', 'Day': '16'}, 'DateRevised': {'Year': '2021', 'Month': '06', 'Day': '16'}, 'Article': DictElement({'ArticleDate': [], 'ELocationID': [StringElement('10.2174/1871527319666201120143654', attributes={'EIdType': 'doi', 'ValidYN': 'Y'})], 'La

In [18]:
extract_abstract.columns

Index(['PMID', 'Title', 'Abstract', 'Journal', 'Language', 'Year', 'Month',
       'Abstract_1', 'Outcomes', 'Outcomes_from_results', 'Study_design'],
      dtype='object')

In [19]:
pmid = papers["PubmedArticle"][0]["MedlineCitation"]["PMID"]

In [20]:
str(pmid)

'33222680'

In [21]:
link = f"https://pubmed.ncbi.nlm.nih.gov/{str(pmid)}"